## 1D PIC (evolution_version) two-stream instabilities

Here we compute the 1D version of PIC. The distribution we use corresponds to a particle current going in both directions with the same mean density and opposite velocities. This configuration generates an instibility and we observe it. 

The Electric field is computed from Maxwell's equations (the time derivative of E is used), starting with an initial data for E using $\nabla \cdot \vec{E} = \rho$ (computing the potential and taking the derivative). 

In [ ]:
#using DifferentialEquations
using Plots
using Statistics
using FFTW
FFTW.set_provider!("mkl")
import Pkg; Pkg.add("FileIO")
using FileIO
using Distributions 


Helper functions are defined in this file.

In [ ]:
include("aux_functions.jl")

### Parameters

    L;               Domain of solution 0 <= x <= L (in Debye lengths)
    N;               Number of electrons
    J;               Number of grid points
    vb;              Beam velocity
    dt;              Time-step (in inverse plasma frequencies)
    t_f;             Simulation run from t = 0. to t = t_f
    order;           smoothnes of the particle shape function


In [ ]:
const order = 2
const L = 100
#N = 200000
#N = 80000
#N = 40000
const N = 20000
#J = 2048
#J = 1024
const J = 128
#J = 254
#J = 512
vb = 3
dt = 0.1
t = 0.0
t_f = 40.0
M = 401
M_g = 20 + 1 #number of outputs, starting from the initial data
dt = t_f / (M-1)
t_i = 0.0
#M = convert(Int64,t_f/dt)
#M=1
κ = 2π/L # for Fourier Transform
dx = L/J
x = [dx*(i-1) for i in 1:J] ;
p = (L, N, J, κ, dx, order)

### Produce the intial data

The 1D distribution for $\frac{m}{kT} = 1$ is just the normal distribution centered at both $\pm v_p$. With $\sigma = 1$.

**Important** the velocity distribution must be symmetrical, otherwise the resulting $v_{total}$ and $S_{total}$ do not vanish to a degree that causes problems.

In [ ]:
E = zeros(J)
ϕ = zeros(J)
n = zeros(J) #charge density
S = zeros(J) #carge current
du = zeros(2*N+J); # contains r, v and E


We take some distributions and fix them to use several times for initial data. We can do here, uncommenting the lines below, or we can do in the Initial_Distributions notebook. We preffer the second way so that we can try the same distribution with different simulation parameters.

In [ ]:
#r = L*rand(N);
#vp = rand(Normal(vb,1),N÷2);
#par_dis = [r; -vp; vp];

In [ ]:
#u0 = copy(u); # make a copy of the initial data so as to use it several times.
par_dis = load("Initial_Distributions/par_dis_L100_N2_5_vp_3.jld2", "par_dis");
#par_dis = load("Initial_Distributions/par_dis_L100_N4_5_vp_3.jld2", "par_dis");
#par_dis = load("Initial_Distributions/par_dis_L100_N8_5_vp_3.jld2", "par_dis");

@assert length(par_dis) ÷ 2 == N


With the particle distribution we construct the initial data for the electric field. For that first we compute de density, using different types of particles (order).

In [ ]:

get_density!(par_dis, n, p)
n0 = N/L
#get_ϕ!(ϕ, -n/n0 .+ 1., κ) # chenge the sign here to make it consistent with charge conservation and the time derivative of E
get_ϕ!(ϕ, n/n0 .- 1., κ)
get_E_from_ϕ!(ϕ,E,dx)
u = [par_dis;E];
length(u)

### What follows are characterizations of the initial data properties.

In [ ]:

get_current!(u, S, p)

Coordinate_test(u[1:N],L)

println("n_total = $(sum(n .- n0))")
println("v_total = $(sum(u[N+1:2N]))")
println("E_total = $(sum(E))")

println("S_total = $(sum(S))")


In [ ]:
#plt = scatter(u[1:N],u[N+1:2N], thickness_scaling = 0.3
#, markersize = 0.3
#, title = "phase-space", legend =:false)
#png("t00_$(J)_2_4")

Initialize some of the intermediate fields we need.

Compute the initial density, potential and electric field.

In [ ]:
plot(layout=(2,2))
plot!(subplot=1,x,n/n0, title = "density", legend = :false)
plot!(subplot=2,x,ϕ, title = "potential", legend = :false)
plot!(subplot=3,x,E, title = "Electric Field", legend = :false)

Find the correspondig potential

Check the histograms for the initial positions and velocities.

In [ ]:
plot(layout=(2,2))
histogram!(subplot=1,u[1:N], title = "density", legend = :false)
histogram!(subplot=2,u[N+1:2*N], title = "velocity", legend = :false)
get_current!(u, S, p)
histogram!(subplot=3,S, title = "current", legend = :false)

## TIME Evolution: 

Now we make the time evolution:

1. we define parameters and vectors where to upload data to study the time evolution: Kinetic Energy, Electric field Energy, Total Electric field, total velocity (momentum), total density, total charge current.

2. we save a vector with the particle phase space, this can be a lot of memory!


In [ ]:
#u = u0
p_RHSC = (N, J, L, dx, order, n, S, du, get_density!, get_current!, Interpolate_2) ;


Energy_K = zeros(M_g)
Energy_E = zeros(M_g)
E_T = zeros(M_g)
v_T = zeros(M_g)
D_T = zeros(M_g)
S_T = zeros(M_g)
T = zeros(M_g)
par = zeros(M_g,2N)
Energy_K[1], Energy_E[1]  = get_energy(u,(L, N, J))
E_T[1] = sum(u[2N+1:end])
v_T[1] = sum(u[N+1:2N])
get_density!(u, n, p)
get_current!(u, S, p)
D_T[1] = sum(n)/n0/J - 1
S_T[1] = sum(S)/n0/J
T[1] = var(u[N+1:2N])
par[1,:] = u[1:2N];



In [ ]:


t = 0.0
j = 1
for k in 2:(M+1)
  RK4_Step!(RHSC,u,t,dt,p_RHSC)
  u = [make_periodic!(u[1:N],L); u[N+1:end]]
  filter_constant!(u[2N+1:end])
  t = t + dt
  if (k-1) % (M÷(M_g-1)) == 0
    j = j+1
    #scatter(plt, u[1:N], u[N+1:2*N])
    Energy_K[j], Energy_E[j] = get_energy(u,(L, N, J))
    E_T[j] = sum(u[2N+1:end])
    v_T[j] = sum(u[N+1:2N])
    get_density!(u, n, p)
    get_current!(u, S, p)
    D_T[j] = sum(n)/n0/J - 1
    S_T[j] = sum(S)/n0/J
    T[j] = var(u[N+1:2N])
    println("j = $j , t = $t, k = $k")
    par[j,:] = u[1:2N]
  end
end

## Studying the solutions: 

### Energy:

In [ ]:
plot(abs.(Energy_K[2:end] .- Energy_K[1]), title = "Energy conservation", label = "Kinetic Energy")
plot!(abs.(Energy_E[2:end] .- Energy_E[1]), label = "|Electric Energy|")
plot!(abs.(Energy_K[2:end]  + Energy_E[2:end] .- (Energy_K[1]+Energy_E[1])) ./ (Energy_K[1]+Energy_E[1]) , yscale=:log10, label = "Total Energy / Initial Energy -1 ")
#png("energy_conservation_T$(t_f)_8_5_$(J)_o$(order)")

In [ ]:
DEdE = abs.(Energy_K[end]  + Energy_E[end] .- (Energy_K[1]+Energy_E[1])) ./ (Energy_K[1]+Energy_E[1])
println("Energy change = $(DEdE)")

### Temperature

In [ ]:
plot(T)

In [ ]:
println("Final temperature = $(T[end])")

## Total Quantities

In [ ]:
plot(layout=(2,2))
plot!(subplot=1,E_T,title="Total Electric Field")
plot!(subplot=2,v_T./N, title="Total velocity")
plot!(subplot=3,D_T,title = "Total density")
plot!(subplot=4,S_T,title = "Total Current")


In [ ]:
E_F = zeros(J)
ϕ_F = zeros(J)
n_F = zeros(J)
get_density!(u, n_F, p)
n0 = N/L
get_ϕ!(ϕ_F, n_F/n0 .+ 1, κ)
#get_ϕ!(ϕ_F, n_F .- n0, κ)
get_E_from_ϕ!(ϕ_F,E_F,dx)
println("averaged total E field = $(sum(E_F))")
println("E_total = $(sum(u[2N+1:end])/J)")
println("Total velocity = $(sum(u[N+1:2N])/N)")
println("Total Charge = $(sum(n_F .- n0))")
println("Final Energy = $(get_energy(u,(L, N, J)))")
get_current!(u, S, p)
println("Total_current = $(sum(S)/J)")

|t_f = 40, Interpolation_1  | |
|--------------------------|--|
|averaged total E field | -9.159339953157541e-16 |
|E_total  |-0.0003034171290374146 |
|Total velocity | 0.00012033529851105414 |
|Total Charge | -1.4495071809506044e-12 |
|Final Energy | (95666.07007308169, 4633.002504126332) |
|Total_current | 0.024067059702236576 |
|Constraint | 0.010031818096308 |
|Energy change | 1.7402810260691252e-5 |
|Final temperature | 9.572090738530141 |


|t_f = 40, Interpolation_2  | |
|--------------------------|--|
|averaged total E field | 7.494005416219807e-16 |
|E_total | -0.00030310449062972825 |
|Total velocity | 0.00015129620185743987 |
|Total Charge | -1.8758328224066645e-12 |
|Final Energy | (95722.30031857773, 4591.237628590617) |
|Total_current | 0.030259240371492278 |
|Constraint | 0.009790360640070031 |
|Energy change | 0.0001650026355367839  |
|Final temperature | 9.577702463840646 |


We don't see much change among both interpolation schemes.

### Statistical quantities

In [ ]:
get_density!(u,n,p)
get_current!(u,S,p)
plot(layout=(2,2))
plot!(subplot=1,x,n*L/N .- 1, title = "density", legend =:false)
plot!(subplot=2,x,S*L/N, title = "current", legend =:false)
histogram!(subplot=3,u[1:N], title = "density", legend =:false)
histogram!(subplot=4,u[N+1:2N], title = "velocity", legend =:false)

### Final Particle Configuration

In [ ]:
scatter(u[1:N],u[N+1:2N]
#, thickness_scaling = 0.3
, markersize = 0.3
, title = "phase-space", legend =:false)
#png("t40_256_4_5_o$(order)_ps")

### Constraint propagation:

We now look at constraint propagation. We compute the electric field from the constraint and compare with the one from the evolution.

In [ ]:

plot(x,u[2N+1:end], label = "dynamical", title="Electric Field with order = $(order)")
plot!(x,E_F,label="from constraint", ls=:dash)
#t_f = 40
#png("t$(t_f)_$(J)_8_5_o$(order)_E")


In [ ]:
plot(x,u[2N+1:end]-E_F, label = "difference", title="Electric Field")
#png("electric_diff_t$(t_f)_$(J)_8_5_o$(order)")

In [ ]:
constraint = sum(abs.(u[2N+1:end]-E_F))/J
println("Constraint = $(constraint)")

$|E_d - E_c|/J$ 

|  order |  N     |  J  | t |  constraint    | $\Delta E$ / E  |
|--------|--------|-----|----|-------|-----------------|
|     1  | $8\; 10^5$ |  512| 20 | 0.036849  |   $10^{-3}$|
|  2 | $8 \;10^5$  |  512| 20  |0.023689   | 0.000494  |
|  3 |  $8 \;10^5$ | 512| 20   | 0.016713  | 0.0003277  |
|  4 | $8 \;10^5$  |  512| 20  | 0.01221  | 0.0002313  |
|  5 | $8 \;10^5$  | 512| 20   | 0.00915  |  0.000170745 |
|  5 | $8 \;10^5$  | 512| 40   |  0.015966 |  0.000231 |
|  5 | $4 \;10^5$  |  256 | 40  | 0.00359  | 5.25e-5 |
|  5 | $8 \;10^5$  | 256  | 40  | 0.00255  | 2.48e-5|
|   |   |   |   |   |


### TVD

We next study who rought is the solution. This is important to compute the derivatives of the electric and magnetic fields for doing full 3-D evolution.

In [ ]:
sum(abs.(u[2N+2:end] - u[2N+1:end-1]))

TVD: 

L = 100, J = 512

N = 40000 

1. for order = 2 we get: 36.531276883274664
2. for order = 5 we get: 18.208051437913298
3. order = 5, J = 256: 13.13132

N = 80000

1. for order = 2 we get: 27.96514399036865
2. for order = 5 we get: 15.360813711818896
3. order = 5, 265, 12.078546053176117


### For animations

In [ ]:
anim = @animate for i = 1:M_g
    scatter(par[i,1:N], par[i,N+1:2N]
    , markersize = 0.3
    , title = "phase-space"
    , legend=false
    , ylim = (-6,6)
    , xlim = (0,100)
    )
end
 
gif(anim, "particles_8_5_256_40_fps5.gif", fps = 5)